# Concurrency and async/await

Reference: [FastAPI docs](https://fastapi.tiangolo.com/async/#is-concurrency-better-than-parallelism)

If you are using third party libraries that tell you to call them with `await`, like:

`results = await some_library()`


Then, declare your path operation functions with `async def` like:
```python
@app.get('/')
async def read_results():
    results = await some_library()
    return results
```

**Note**: You can only use `await` inside of functions with `async def`.

- When using a third-party library that interacts with external systems (e.g., a database, an API, the file system).
- If the library doesn't support `await` (common with many database libraries).
- Declare your path operation functions with a regular `def` instead of `async def`.
```python
@app.get('/')
def results():
    results = some_library()
    return results
```

- If your application (somehow) doesn't have to communicate with anything else and wait for it to respond, use `async def`.
- If you just don't know, use normal `def`.

**Note**:

- You can mix `def` and `async def` in your path operation functions as needed.
- Define each function using the most appropriate option for your use case.
- FastAPI will handle both `def` and `async def` functions correctly.
- FastAPI will continue to operate asynchronously and maintain high performance.
- Following these guidelines allows FastAPI to optimize performance further.

### Technical Details
Recent versions of Python support writing asynchronous code through a concept called "coroutines." Coroutines allow for non-blocking operations, meaning that while waiting for certain tasks (like I/O operations) to complete, other code can continue running.

**Asynchronous Code**
- Asynchronous code allows a program to handle tasks that require waiting (e.g., for I/O operations) without blocking execution.
- During waiting, the program can perform other tasks rather than doing nothing.
- The program periodically checks if any of the waiting tasks have completed.
- Once a task finishes (e.g., reading a file, receiving network data), the program processes it.
- Common "slow" tasks include network operations, file I/O, and database queries.
- These operations are "I/O bound" and can be performed asynchronously to improve efficiency.
- Asynchronous programming avoids waiting idly for tasks to finish and allows the program to manage multiple tasks more effectively.
- Synchronous (or sequential) programming executes tasks in a strict order, blocking until each task completes before moving on.


### Concurrency + Parallelism: Web + Machine Learing
- FastAPI leverages concurrency, a common feature in web development, similar to NodeJS.
- It also supports parallelism and multiprocessing for CPU-bound tasks, such as those in Machine Learning.
- Python’s prominence in Data Science, Machine Learning, and Deep Learning enhances FastAPI’s suitability for Data Science and Machine Learning web APIs and applications.
- This combination makes FastAPI a strong choice for building data-intensive and machine learning applications.

### More technical details
- `await` can only be used inside functions defined with `async def`.
- Functions defined with `async def` must be "awaited" and can only be called inside other `async def` functions.
- This creates a circular dependency: how do you call the first `async` function?
- With FastAPI, you don't need to worry about this because your path operation function handles it, and FastAPI manages the execution.
- If you're using async/await outside of FastAPI, you can still implement it by ensuring that all async functions are called within other async functions.

### Coroutines
- A **coroutine** is the term for the object returned by an `async def` function.
- It is like a function that Python can start and will eventually finish, but it may pause internally due to `await`.
- The use of `async` and `await` is often summarized as using "coroutines."
- This concept is similar to "Goroutines" in Go, which is a key feature of the Go language.

### Path operation functions
- When using `def` for a path operation function instead of `async def`, the function is executed in an external threadpool and awaited, rather than running directly (to avoid blocking the server).

- If you're transitioning from another async framework where using `def` for trivial compute-only functions provided a slight performance gain, this approach in FastAPI might have the opposite effect.

- In FastAPI, it is generally better to use `async def` for path operation functions unless they involve blocking I/O operations.

- Regardless of the approach, FastAPI is likely to be faster than or at least comparable to your previous framework.

### Dependencies
- For dependencies in FastAPI, if you use a standard `def` function instead of `async def`, the dependency will be executed in an external threadpool.
- This approach ensures that synchronous dependencies do not block the main event loop of the server.

### Sub-Dependencies
- FastAPI supports having multiple dependencies and sub-dependencies that require each other as parameters in function definitions.
- Dependencies can be created with either `async def` or `def`.
- Dependencies created with `async def` will be awaited, while those created with `def` will be executed on an external thread from the threadpool.
- This mixed approach ensures that all dependencies function correctly, with `def` dependencies running asynchronously but not blocking the main event loop.

### Other Utility Functions
- Utility functions that you call directly in your code can be created with either `def` or `async def` without any impact from FastAPI.
- If a utility function is defined with `def`, it will be called directly as written in your code.
- If a utility function is defined with `async def`, you should use `await` when calling it in your code.
- This is different from path operation functions and dependencies, which FastAPI manages for you. For these, synchronous functions are handled in a threadpool, while asynchronous functions are awaited.